In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [7]:
data = pd.read_csv('sgtf_regionepicurve_2021-12-30.csv')
data.head()

,UKHSA_region,specimen_date,n,percent,sgtf,total,conf_low,conf_high
0,East Midlands,01/11/2021,874,100.00,Cases with confirmed S-gene,874,99.454454,100.00000
1,East Midlands,02/11/2021,724,99.86,Cases with confirmed S-gene,725,99.109120,99.99280
2,East Midlands,02/11/2021,1,0.14,Cases with confirmed SGTF,725,0.007200,0.89088
3,East Midlands,03/11/2021,757,100.00,Cases with confirmed S-gene,757,99.370567,100.00000
4,East Midlands,04/11/2021,598,100.00,Cases with confirmed S-gene,598,99.204289,100.00000


In [8]:
y = data.loc[:,['sgtf']]

data_data = LabelEncoder()

data['UKHSA_region_data'] = data_data.fit_transform(data['UKHSA_region'])
y['sgtf_data'] = data_data.fit_transform(y['sgtf'])


#y['sgtf_data'] = data_data.fit_transform(y['sgtf'])
y = y.drop(['sgtf'],axis='columns')


data = data.drop(['UKHSA_region','sgtf'],axis='columns')
print(data.head(0))
print(y.head(0))


Empty DataFrame
Columns: [specimen_date, n, percent, total, conf_low, conf_high, UKHSA_region_data]
Index: []
Empty DataFrame
Columns: [sgtf_data]
Index: []


In [9]:
y.drop_duplicates(subset=['sgtf_data'])

,sgtf_data
0,0
2,1


In [10]:
data = data.set_index('specimen_date')
data.index = pd.to_datetime(data.index,unit='ns')
print(data.index)

DatetimeIndex(['2021-01-11', '2021-02-11', '2021-02-11', '2021-03-11',
               '2021-04-11', '2021-05-11', '2021-05-11', '2021-06-11',
               '2021-07-11', '2021-08-11',
               ...
               '2021-12-24', '2021-12-24', '2021-12-25', '2021-12-25',
               '2021-12-26', '2021-12-26', '2021-12-27', '2021-12-27',
               '2021-12-28', '2021-12-28'],
              dtype='datetime64[ns]', name='specimen_date', length=943, freq=None)


C:\Users\Ajinkya\AppData\Local\Temp\ipykernel_8740\3824577259.py:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  data.index = pd.to_datetime(data.index,unit='ns')


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
import numpy as np
from sklearn.metrics import accuracy_score
for i in np.arange(0,1,1):
    X_train,X_test,y_train,y_test = train_test_split(data,y,test_size=0.02,shuffle=False)
    models = ExtraTreesClassifier(n_estimators=10,random_state=20,criterion="gini")
    models.fit(X_train,y_train)
    predIdxs = models.predict(X_test)
    print("X: ",i,accuracy_score(y_test,predIdxs))

X:  0 1.0


C:\Users\Ajinkya\AppData\Local\Temp\ipykernel_8740\3271403610.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  models.fit(X_train,y_train)


In [12]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import *
cm = confusion_matrix(y_test, predIdxs)
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
print("Confusion Matrix: \n",cm)
print("accuracy: {:.4f}".format(acc))
print("specificity: {:.4f}".format(specificity))
print("F1: {:.4f}".format(f1_score(y_test,predIdxs)))

Confusion Matrix: 
 [[ 9  0]
 [ 0 10]]
accuracy: 1.0000
specificity: 1.0000
F1: 1.0000


In [13]:
prediction = [[1],[0.14],[725],[0.007200],[0.890879771],[0]]
prediction = np.reshape(prediction, (1,6))

In [14]:
pred = models.predict(prediction)

if pred[0] == 0:
    print("Cases with confirmed S-gene!")
elif pred[0] == 1:
    print("Cases with confirmed SGTF!")
else:
    print("Didn't find confirmed!")

Cases with confirmed SGTF!


C:\Users\Ajinkya\Kobe\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but ExtraTreesClassifier was fitted with feature names
  warnings.warn(


In [15]:
from sklearn.metrics import roc_curve
from sklearn import metrics
fpr, tpr, thresholds = roc_curve(y_test,predIdxs)
#AUC SCORE
(metrics.auc(fpr, tpr))*100

100.0